## MVP – Apontamento de OP por Voz (Whisper + gTTS + CSV):
Este notebook cria um MVP para apontamento de Ordens de Produção (OP) por voz.
Fluxo: o usuário fala ex: “Apontar OP 1015”, o sistema transcreve com Whisper, extrai o número da OP, pede confirmação (“sim” ou “não”), e registra o resultado em um arquivo CSV.

In [202]:
# Célula 01 — Define o idioma padrão (português)
language = "pt"


In [203]:
# Célula 02 — Grava áudio pelo navegador no Colab usando MediaRecorder (JavaScript) e salva em arquivo

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5, file_name="audio.webm"):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f"/content/{file_name}"


In [204]:
# Célula 03 — Testa a gravação e permite ouvir o áudio capturado
print("Ouvindo... Fale: 'Apontar OP 1015'")
record_file = record(sec=5, file_name="cmd_audio.webm")

print("Arquivo salvo em:", record_file)
display(Audio(record_file, autoplay=False))


Ouvindo... Fale: 'Apontar OP 1015'


<IPython.core.display.Javascript object>

Arquivo salvo em: /content/cmd_audio.webm


In [205]:
# Célula 04 — Instala o Whisper (open-source)
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [206]:
# Célula 05 — Carrega o modelo do Whisper (quanto maior, melhor e mais pesado)
import whisper

model = whisper.load_model("small")  # você pode trocar para "medium" se quiser mais qualidade


In [207]:
# Célula 06 — Transcreve o comando de voz ("Apontar OP 1015") para texto
result = model.transcribe(
    record_file,
    fp16=False,
    language=language,
    initial_prompt="Comandos industriais: apontar OP, ordem de produção. Exemplos: OP 1015, OP 2050."
)
transcription = (result.get("text") or "").strip()

print("TRANSCRIÇÃO DO COMANDO:")
print(transcription)


TRANSCRIÇÃO DO COMANDO:
Apontar OP 1020.


In [208]:
# Célula 07 — Instala o gTTS (Text-to-Speech) para o sistema falar por voz
!pip install -q gTTS


In [209]:
# Célula 08 — Funções principais:
# - norm: normaliza texto
# - extract_op: extrai o número da OP
# - speak: converte texto em voz (mp3)
# - is_yes/is_no: identifica sim ou não
# - log_apontamento: salva o resultado em CSV

import re
import unicodedata
import uuid
import csv
import os
from datetime import datetime
from gtts import gTTS
from IPython.display import Audio, display

def norm(s: str) -> str:
    s = (s or "").strip().lower()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")  # remove acentos
    s = re.sub(r"[^a-z0-9\s]", " ", s)   # remove pontuação
    s = re.sub(r"\s+", " ", s).strip()   # normaliza espaços
    return s

def extract_op(text: str):
    t = norm(text)

    # OP / O P / ORDEM + número
    m = re.search(r"\b(?:op|o\s*p|ordem)\s*#?\s*(\d{1,10})\b", t)
    if m:
        return m.group(1)

    # fallback: se tiver contexto, pega um número
    if any(k in t for k in ["apont", "ponta", "aponta", "ordem", "op", "o p"]):
        m = re.search(r"\b(\d{3,10})\b", t)
        if m:
            return m.group(1)

    return None

def speak(text: str, lang: str = "pt"):
    out_file = f"/content/tts_{uuid.uuid4().hex}.mp3"
    gTTS(text=text, lang=lang, slow=False).save(out_file)
    display(Audio(out_file, autoplay=True))
    return out_file

def is_yes(text: str) -> bool:
    t = norm(text)
    return re.search(r"\b(sim|ok|confirmo|confirmar|pode|positivo)\b", t) is not None

def is_no(text: str) -> bool:
    t = norm(text)
    return re.search(r"\b(nao|cancelar|cancela|negativo)\b", t) is not None

def log_apontamento(op: str, status: str, cmd_text: str, confirm_text: str):
    path = "/content/apontamentos.csv"
    exists = os.path.exists(path)
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if not exists:
            w.writerow(["timestamp", "op", "status", "comando_transcrito", "confirmacao_transcrita"])
        w.writerow([datetime.now().isoformat(), op, status, cmd_text, confirm_text])
    return path


In [210]:
# Célula 09 — Extrai OP, pede confirmação e espera 5 segundos antes de seguir (para usar com "Executar tudo")

import time

print("TRANSCRIÇÃO BRUTA:", transcription)
print("NORMALIZADA:", norm(transcription))

cmd_text = transcription
op = extract_op(cmd_text)

print("OP extraída:", op)

if not op:
    msg = "Não consegui identificar a OP. Por favor diga: apontar OP e o número."
    print(msg)
    speak(msg, lang=language)
    awaiting_confirmation = False
else:
    msg = f"Você gostaria de apontar a OP {op}? Responda sim ou não."
    print(msg)
    speak(msg, lang=language)

    # guarda o que a próxima célula vai usar
    awaiting_confirmation = True
    pending_op = op
    pending_cmd_text = cmd_text

    # pausa REAL de 5 segundos para você se preparar
    print("Aguarde 5 segundos... Em seguida vou ouvir sua resposta (sim/não).")
    time.sleep(5)


TRANSCRIÇÃO BRUTA: Apontar OP 1020.
NORMALIZADA: apontar op 1020
OP extraída: 1020
Você gostaria de apontar a OP 1020? Responda sim ou não.


Aguarde 5 segundos... Em seguida vou ouvir sua resposta (sim/não).


In [211]:
# Célula 10 — Grava confirmação (sim/não), transcreve, decide e salva no CSV
# (Sem TTS antes da gravação, para não "vazar" o áudio do sistema no microfone)

from IPython.display import Audio, display
import time

if not globals().get("awaiting_confirmation", False):
    print("Nada para confirmar. Rode a gravação + transcrição + Célula 09 primeiro.")
else:
    op = pending_op
    cmd_text = pending_cmd_text

    print("🎤 GRAVANDO AGORA! Responda: SIM ou NÃO")
    # (opcional) um mini atraso de 0.3s só para você ver a mensagem antes de começar
    time.sleep(0.3)

    confirm_file = record(sec=4, file_name="confirm_audio.webm")
    display(Audio(confirm_file, autoplay=False))

    result2 = model.transcribe(
        confirm_file,
        fp16=False,
        language=language,
        initial_prompt="Respostas curtas: sim, não, cancelar, confirmar."
    )
    confirm_text = (result2.get("text") or "").strip()

    print("Confirmação transcrita:", confirm_text)
    print("NORMALIZADA:", norm(confirm_text))
    print("YES?", is_yes(confirm_text), "| NO?", is_no(confirm_text))

    if is_yes(confirm_text):
        status = "APONTADO"
        final_msg = f"OP {op} apontada com sucesso."
    elif is_no(confirm_text):
        status = "CANCELADO"
        final_msg = "Apontamento cancelado."
    else:
        status = "INDEFINIDO"
        final_msg = "Não entendi. Diga apenas sim ou não."

    print(final_msg)
    speak(final_msg, lang=language)

    csv_path = log_apontamento(op, status, cmd_text, confirm_text)
    print("CSV salvo em:", csv_path)

    # reseta o estado
    awaiting_confirmation = False
    pending_op = None
    pending_cmd_text = None


🎤 GRAVANDO AGORA! Responda: SIM ou NÃO


<IPython.core.display.Javascript object>

Confirmação transcrita: Sim.
NORMALIZADA: sim
YES? True | NO? False
OP 1020 apontada com sucesso.


CSV salvo em: /content/apontamentos.csv


In [212]:
import pandas as pd
pd.read_csv("/content/apontamentos.csv")


,timestamp,op,status,comando_transcrito,confirmacao_transcrita
0,2026-02-03T00:38:45.397858,1015,INDEFINIDO,Apontar OP 1015.,Sim.
1,2026-02-03T00:39:36.928645,1015,INDEFINIDO,Apontar OP 1015.,Sim.
2,2026-02-03T00:39:47.942704,1015,INDEFINIDO,Apontar OP 1015.,Sim.
3,2026-02-03T00:40:05.779093,1015,INDEFINIDO,Apontar OP 1015.,Não.
4,2026-02-03T00:45:24.319049,1015,INDEFINIDO,Apontar OP 1015.,NaN
5,2026-02-03T00:46:31.903220,1015,INDEFINIDO,Apontar OP 1015.,NaN
6,2026-02-03T01:01:01.662383,1015,INDEFINIDO,Apontar OP 1015.,NaN
7,2026-02-03T01:06:26.144401,1015,INDEFINIDO,Apontar OP 1015.,NaN
8,2026-02-03T01:07:13.963335,1015,INDEFINIDO,Apontar OP 1015.,NaN
9,2026-02-03T01:08:46.096378,1015,INDEFINIDO,Apontar OP 1015.,NaN
